In [2]:
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [8, 8]

from faker import Factory
from itertools import islice
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle

import pyodbc
import pysal as ps

from pysal.viz.mapclassify import Natural_Breaks as nb

from math import log10
from shapely.geometry import Point
from shapely import wkt

from IPython.display import IFrame

import folium
from folium.plugins import HeatMap

fake = Factory.create()

c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [6]:
# LIMITED READ

conn = pyodbc.connect('DSN=GEODWH;UID=GEODWH_READ;PWD=readonly')
# ss_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[SAMPLEID],[SAMPCODE],[FILE_ID],[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_SURFSAMP_TRACE] WHERE [LNG94] > 140 and [LNG94] < 142 and [LAT94] > -32.5 and [LAT94] < -31 and SAMPCODE in ('SOIL', 'AUGER', 'SURF_DRILL', 'LAG', 'FLOAT', 'UNKNOWN','VEGETATION')"
# dh_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[HOLEID] AS SAMPLEID,[DRILLCODE] AS SAMPCODE,[ASSAY_FILE_ID] AS FILEID,[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_MAX_DRILL_TRACE] WHERE DRILLCODE in ('AUGER', 'VAC', 'SURF_DRILL', 'VIB', 'AUG') and [LNG94] > 140 and [LNG94] < 142 and [LAT94] > -32.5 and [LAT94] < -31"

# # Read the Auger samples from the drillhole table
# # We only use : 'SOIL', 'AUGER', 'SURF_DRILL', 'LAG', 'FLOAT', 'UNKNOWN','VEGETATION'

# drillholes = pd.read_sql(dh_select, conn)

In [ ]:
# conn = pyodbc.connect('DSN=GEODWH;UID=GEODWH_READ;PWD=readonly')

# ss_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[SAMPLEID],[SAMPCODE],[FILE_ID],[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_SURFSAMP_TRACE] WHERE SAMPCODE in ('SOIL', 'AUGER', 'SURF_DRILL', 'LAG', 'FLOAT', 'UNKNOWN','VEGETATION')"
# dh_select = "SELECT [RPT_ID],[RIN],[RPT_TYPE],[HOLEID] AS SAMPLEID,[DRILLCODE] AS SAMPCODE,[ASSAY_FILE_ID] AS FILEID,[LAT94],[LNG94],[As_ppm],[Au_ppm] FROM [GEODWH].[dbo].[ASSAY_MAX_DRILL_TRACE] WHERE DRILLCODE in ('AUGER', 'VAC', 'SURF_DRILL', 'VIB', 'AUG')"


In [ ]:
# # Read from the database
# dh = pd.read_sql(dh_select, conn)
# ss = pd.read_sql(ss_select, conn)

In [10]:
# confidence = "SELECT [MR_AUTHOR], [MR_CONFIDENTIALITY],[MR_RECTYPE],[MR_REPYEAR],[MR_RIN] FROM [GEODWH].[dbo].[DG_MINREPS] where MR_RECTYPE = 'MINERAL'"

# rpt_confidence = pd.read_sql(confidence, conn)
# rpt_confidence.head()

# # Pickle them up for next time
# dsfolder = os.path.join(os.path.dirname(os.getcwd()), 'data_src')

# with open(os.path.join(dsfolder, 'rpt_confidence.pickle'), 'wb') as f:
#     pickle.dump(rpt_confidence, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# # Pickle them up for next time
# dsfolder = os.path.join(os.path.dirname(os.getcwd()), 'data_src')

# with open(os.path.join(dsfolder, 'dh.pickle'), 'wb') as f:
#     pickle.dump(dh, f, pickle.HIGHEST_PROTOCOL)
    
# with open(os.path.join(dsfolder, 'ss.pickle'), 'wb') as f:
#     pickle.dump(ss, f, pickle.HIGHEST_PROTOCOL)

In [11]:
dsfolder = os.path.join(os.path.dirname(os.getcwd()), 'data_src')

# Read the pickles
with open(os.path.join(dsfolder, 'drillholes.pickle'), 'rb') as fin:
    dh = pickle.load(fin)
    
with open(os.path.join(dsfolder, 'ss.pickle'), 'rb') as fin:
    ss = pickle.load(fin)
    
with open(os.path.join(dsfolder, 'rpt_confidence.pickle'), 'rb') as fin:
    rpt_confidence = pickle.load(fin)

In [12]:
# Check unique sample codes
dhtypes = dh['SAMPCODE'].unique()
dh['SAMPCODE'].replace('AUG', 'AUGER', inplace=True) # Fix a namecase
dh['SAMPCODE'].replace('Auger', 'AUGER', inplace=True) # Fix a namecase
dh['SAMPCODE'].replace('auger', 'AUGER', inplace=True) # Fix a namecase

In [13]:
dhtypes = dh['SAMPCODE'].unique()
print(dhtypes)

['AUGER' 'SURF_DRILL' 'VIB' 'VAC']


In [14]:
ss['SAMPCODE'] = ss['SAMPCODE'].map(lambda x: x.strip())# Fix some whitespaces
ss['RIN'] = ss['RIN'].map(lambda x: x.strip())
ss['RPT_TYPE'] = ss['RPT_TYPE'].map(lambda x: x.strip())
ss['SAMPLEID'] = ss['SAMPLEID'].map(lambda x: x.strip())

In [15]:
ss['SAMPCODE'].replace('Soil', 'SOIL', inplace=True)
ss['SAMPCODE'].replace('soil', 'SOIL', inplace=True)

In [16]:
sstypes = ss['SAMPCODE'].unique()
print(sstypes)

['SOIL' 'SURF_DRILL' 'FLOAT' 'UNKNOWN' 'LAG' 'AUGER' 'VEGETATION']


In [17]:
# Combine the drilling with the surface samples
samples = ss.append(dh)

# Check unique sample codes
stypes = samples['SAMPCODE'].unique()
stypes.sort()

print(f'Sample types: {stypes}')
print(len(samples))

c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Sample types: ['AUGER' 'FLOAT' 'LAG' 'SOIL' 'SURF_DRILL' 'UNKNOWN' 'VAC' 'VEGETATION'
 'VIB']
700554


In [18]:
dsfolder = os.path.join(os.path.dirname(os.getcwd()), 'data_src')

with open(os.path.join(dsfolder, 'samples.pickle'), 'wb') as f:
    pickle.dump(samples, f, pickle.HIGHEST_PROTOCOL)

In [20]:
with open(os.path.join(dsfolder, 'samples.pickle'), 'rb') as f:
    samps = pickle.load(f)

In [21]:
# Take a look

print(len(samps))

700554


In [22]:
samps.head()

,As_ppm,Au_ppm,FILEID,FILE_ID,LAT94,LNG94,RIN,RPT_ID,RPT_TYPE,SAMPCODE,SAMPLEID
0,2.4,0.002,NaN,286.0,-31.860818,141.665005,R00029271,28,A,SOIL,370056
1,2.1,0.003,NaN,286.0,-31.860389,141.665091,R00029271,28,A,SOIL,370057
2,3.3,0.023,NaN,286.0,-31.862621,141.663997,R00029271,28,A,SOIL,370058
3,3.0,0.021,NaN,286.0,-31.862969,141.663766,R00029271,28,A,SOIL,370059
4,3.2,-0.001,NaN,286.0,-31.857557,141.665381,R00029271,28,A,SOIL,370060


In [23]:
samples_full = pd.merge(samps, rpt_confidence, how='inner', left_on='RIN', right_on='MR_RIN', left_index=False, right_index=False)

In [24]:
samples_full.drop(['MR_RIN'], inplace=True, axis=1)
samples_full.drop(['MR_RECTYPE'], inplace=True, axis=1)

In [26]:
from pyproj import Proj
from pyproj import transform

geographic = Proj(init='epsg:4283')
projected = Proj(init='epsg:3112')

def lat(coords):
    E, N = transform(geographic, projected, coords[1], coords[0])
    return E

def lon(coords):
    E, N = transform(geographic, projected, coords[1], coords[0])
    return N

In [27]:
samples_full['EASTING'] = samples_full.apply(lambda x: lat((x['LAT94'], x['LNG94'])), axis=1)
samples_full['NORTHING'] = samples_full.apply(lambda x: lon((x['LAT94'], x['LNG94'])), axis=1)

In [29]:
# Make geometry field wkt (Well Known Text)
samples_full['Coordinates'] = list(zip(samples_full.EASTING, samples_full.NORTHING)) # Round to 6dp
samples_full['Coordinates'] = samples_full['Coordinates'].apply(Point)
samples_full = gpd.GeoDataFrame(samples_full, geometry='Coordinates', crs={'init' :'epsg:3112'})

In [34]:
samples_full.head()

with open(os.path.join(dsfolder, 'samples_conditioned.pickle'), 'wb') as f:
    pickle.dump(samples_full, f, pickle.HIGHEST_PROTOCOL)

In [35]:
with open(os.path.join(dsfolder, 'samples_conditioned.pickle'), 'rb') as f:
    samps = pickle.load(f)

In [ ]:
# Function to Buffer the points

def buffer(data, distance=0.0075, resolution=10):
    """
    Buffer points to return polygons of spatially distinct data sets.
    
    params:
        data: GeoDataFrame
    returns:
        buffer_union: GeoDataFrame of buffers
    """
    
    buffers = data.buffer(distance, resolution)
    buffer_union = buffers.unary_union
    return buffer_union

In [ ]:
# Function to convert the single buffer polygons to a GeoDataFrame

def sb_attributes(singlebuffer):
    """
    Converts wkt polygon to GeoDataFrame
    
    params:
        wkt: Polygon
    returns:
        GeoDataFrame
    """
    colour = fake.hex_color()
    
    # Set a dictonary to build the attributes for the polygon
    sb_attributes = dict()
    sb_attributes['geometry'] = singlebuffer
    sb_attributes['index'] = [0]
    sb_attributes['stroke'] = [colour]
    # TODO add the RIN as a tooltip
    
    # Covert the dictionary into the GeoDataFrame 
    sb_attributes_df = pd.DataFrame(sb_attributes)
    sb_attributes_gdf = gpd.GeoDataFrame(sb_attributes_df)
    sb_attributes_gdf.columns={'geometry': 'geometry', 'index': 'index', 'stroke': 'stroke'}
    sb_attributes_gdf.crs = {'init' :'epsg:4326'}
    
    return sb_attributes_gdf

In [ ]:
# Function to convert the multi buffers polygons to a GeoDataFrame

def mb_attributes(multibuffer):
        """
        Converts wkt MultiPolygon to GeoDataFrame

        params:
            wkt: MultiPolygon
        returns:
            GeoDataFrame
        """
        # Set a dictonary to build the attributes for each polygon
        mb_attributes = dict()
        geometry = []
        index = []
        stroke = []
    
        # Loop over each polygon in the MultiPolygon
        for idx, b in enumerate(multibuffer):
            colour = fake.hex_color()
            stroke.append(colour)
            geometry.append(b)
            index.append(idx)
            # TODO add the RIN as a tooltip

            # Populate the dictionary
            mb_attributes['geometry'] = geometry
            mb_attributes['index'] = index
            mb_attributes['stroke'] = stroke
            
        # Covert the dictionary into the GeoDataFrame 
        mb_attributes_df = pd.DataFrame(mb_attributes)
        mb_attributes_gdf = gpd.GeoDataFrame(mb_attributes_df)
        mb_attributes_gdf.columns={'geometry': 'geometry', 'index': 'index', 'stroke': 'stroke'}
        mb_attributes_gdf.crs = {'init' :'epsg:4326'}
        
        return mb_attributes_gdf

In [ ]:
# Function to join the points to the buffers

def rin_stype_buffered(points, buffers):
    """
    Spatially join points to buffers

    params:
        points: GeoDataFrame containing Points
        buffers: GeoDataFrame containing Points
    returns:
        Combined GeoDataFrame of points and buffers
    """
    points_buffered = gpd.sjoin(points, buffers, how="inner", op='intersects')
    points_buffered.drop(['index_right'], axis=1)
    return points_buffered

In [ ]:
# Peep at the head of the DataFrame
print(f'\nSample count : {len(points)}\n')
print(samples.head())

In [ ]:
# Peep at the gold stats
print(samples['Au_ppm'].describe())
samples['Au_ppm'].hist(range = (0,0.05))

In [ ]:
# Generator function to subdivide the samples according to rin and sampletype

def data_divisions(rins):
    """Divide the data
    
    params:
        Groupby object of samples grouped by rin
    returns:
        yields a tuple (rin, sampleType, sampleData)"""
    # For each rin...
    for rin, rindata in rins:
        stypes = rindata.groupby(['SAMPCODE'])
        
        # For each sample type...
        for stype, rin_stype_data in stypes:
            yield (rin, stype, rin_stype_data)

In [ ]:
# Sort by the rins
samples.sort_values('RIN')

# Groupby the rin
rins = samples.groupby(['RIN'])

# Empty frames to collect the buffers and processed samples
all_buffers = gpd.GeoDataFrame()
sbuffered_samples = gpd.GeoDataFrame()
mbuffered_samples = gpd.GeoDataFrame()
all_buffered_samples = gpd.GeoDataFrame()

# Counter to check the output is valid
total_points = 0

In [ ]:
len(samples)

In [ ]:
# Loop over the samples grouped by the rins and yield the subdivisions
for rin, stype, subsamples in data_divisions(rins):
    
    # We buffer all the samples
    buffers = buffer(subsamples)
    
    print(rin, stype, len(subsamples), buffers.geom_type)
    total_samples = total_samples + len(subsamples)
    if buffers.geom_type == 'Polygon':
        
        # For each of the single wkt polygons convert to a dataframe (adding attributes)
        _df = sb_attributes(buffers)
        assert isinstance(df, gpd.GeoDataFrame)

        # Aggregate the buffers
        all_buffers = gpd.GeoDataFrame(pd.concat([all_buffers, _df], ignore_index=True))
        
        # Spatial join the samples to the buffer
        sbuffered_samples = gpd.sjoin(_df, subsamples, how="inner", op='intersects')
        
        # Aggregate the samples
        all_buffered_samples = gpd.GeoDataFrame(pd.concat([all_buffered_samples, sbuffered_samples], ignore_index=True))

    if buffers.geom_type == 'MultiPolygon':
        
        # Split the Multipolygon wkt to dataframe
        _mdf = mb_attributes(buffers)

        # Set empty frames to collect the subdivided sets
        multisamples = gpd.GeoDataFrame()
        mbuffers = gpd.GeoDataFrame()
        
        # Iterate over the multi-buffer dataframe
        for row, mbuffer in _mdf.groupby('index'):
            assert isinstance(mbuffer, gpd.GeoDataFrame)

            # Aggregate the buffers
            mbuffers = gpd.GeoDataFrame(pd.concat([mbuffers, mbuffer], ignore_index=True))
            
            # Spatially join the samples to the buffers
            mbuffered_samples = gpd.sjoin(mbuffer, subsamples, how="inner", op='intersects')
            
            # Aggregate the samples
            multisamples = gpd.GeoDataFrame(pd.concat([multisamples, mbuffered_samples], ignore_index=True))
        
        # Aggregate the single and multipolygon buffers and samples
        all_buffers = gpd.GeoDataFrame(pd.concat([all_buffers, mbuffers], ignore_index=True))
        all_buffered_samples = gpd.GeoDataFrame(pd.concat([all_buffered_samples, multisamples], ignore_index=True))
        
        # Watch it grow
        print(f'\nCaptured samples: {total_samples} of {len(samples)}\n')

# Assert it worked
assert len(all_buffered_samples) == total_samples == samples

In [ ]:
all_buffers.to_file('ALL_BUFFERS.geojson', driver = "GeoJSON")

In [ ]:
class fix_data(object):
    """"""
    def __init__(self, df, element):
        self.df = df
        self.element = element
    
    def force_numeric(self):
        # Coerce element to numeric values
        self.df[self.element].fillna(0, inplace=True)
        # Coerce element to numeric values
        self.df[f'{self.element}_nn'] = pd.to_numeric(self.df[self.element])

    def neg_conversions(self, x):
        # Define the negative conversions
        if x < -5:
            return 0.0005
        if x < 0:
            return abs(x)/2
        if x == 0:
            return 0.0005
        if x > 0:
            return x
        
    def fix_neg(self):
        self.df[f'{self.element}_fn'] = self.df[self.element].apply(lambda x: self.neg_conversions(x))

    def log(self):
        self.df[f'{self.element}_log'] = self.df[f'{self.element}_fn'].apply(lambda x: log10(x))
        
    def threshold(self):
        median = self.df[f'{self.element}_log'].median()
        # Mean Absolute Deviation: Tukey, J.W., 1977. Exploratory Data Analysis. Addison-Wesley, Reading, 688 pp
        mad = self.df[f'{self.element}_log'].mad()  
        # Set threshold: http://crcleme.org.au/Pubs/guides/gawler/a7_id_anomalies.pdf
        self.threshold = median + 2*mad 
        
    def scaling(self, x):
        # Normalisation of values between 0 and 1
        min_value = self.df[f'{self.element}_log'].min()
        x = (x - min_value) / (self.threshold - min_value) # 'Max' value is the threshold 
        return x
        
    def apply_scaling(self):
        self.df['normalised'] = self.df[f'{self.element}_log'].apply(lambda x: self.scaling(x))

    def natural_breaks(self):
        # Apply hex colours to natural breaks
        classifier = nb(self.df[self.element], 7) # nb are natural breaks
        self.df['classifications'] = self.df[self.element].apply(classifier)
        self.df.classifications.replace([1,2,3,4,5,6,7], ['#82817d','#55b1d9','#5bd955','#e6a94e','#e02d2d','#da2de0','#af00b5'], inplace=True)
        
        

In [ ]:
stats_dict = dict()

In [ ]:
def boxplots(stats_dict, rin, samples):
    vals = samples[f'{element}'].values
    boxplot = mpl.pyplot.boxplot(vals)
    stats_dict[rin] = boxplot
    return stats_dict

In [ ]:
normalised = gpd.GeoDataFrame()

for subset, points in all_buffered_points.groupby(['RIN','index']):
    fd = fix_data(points, 'Au_ppm')
    fd.force_numeric()
    fd.fix_neg()
    fd.log()
    fd.threshold()
    fd.apply_scaling()
    fd.natural_breaks()
    print(len(normalised))
    normalised = gpd.GeoDataFrame(pd.concat([normalised, points], ignore_index=True))
    

In [ ]:
list(normalised)

In [ ]:
normalised.head()

In [ ]:
_gdf = gpd.GeoDataFrame(normalised, crs={'init' :'epsg:4283'})

norm = _gdf[['RIN', 'SAMPCODE', 'index', 'LAT94', 'LNG94', 'Au_ppm', 'normalised', 'classifications', 'stroke']]

norm['normalised'].fillna(0, inplace=True)
norm['classifications'].replace([0], '#82817d', inplace=True)

norm['Coords'] = list(zip(norm.LNG94.round(6), norm.LAT94.round(6)))
norm['Coords'] = norm['Coords'].apply(Point)

In [ ]:
list(normalised)

In [ ]:
samples = list(zip(norm.LAT94.values, norm.LNG94.values, norm.normalised.values, norm.classifications.values))

In [ ]:
samples

In [ ]:
# Show me the buffers !!!!
m = folium.Map(location=[-32, 141.5], zoom_start=9)

samples = folium.FeatureGroup(name="samples")

sample_points = list(zip(norm.LAT94.values, norm.LNG94.values, norm.normalised.values, norm.classifications.values))

for lat, lng, au, colour in sample_points:
    samples.add_child(folium.CircleMarker(location=[lat, lng ], radius=1,
        popup=str(au), 
        tooltip=str(au),
        fill=True,  # Set fill to True
        color=str(colour),
        fill_opacity=0.5)).add_to(m)

folium.GeoJson('ALL_BUFFERS.geojson',
    style_function=lambda x: {
        'color' : 'grey',
        'weight' : 2,
        'opacity': 0.66,
        'fillColor' : x['properties']['stroke'],
        }).add_to(m)

# m.add_child(samples)
m.add_child(folium.LayerControl())
m.save("ALL_SAMPLES.html")

# IFrame(src='ALL_SAMPLES.html', width=1000, height=600)

In [ ]:
subsets = all_buffered_points.groupby(['RIN','index'])

def apply_normalisation(subset):
    fd = fix_data(subset, 'Au_ppm')
    fd.force_numeric()
    fd.fix_neg()
    fd.log()
    fd.threshold()
    fd.apply_scaling()
    fd.natural_breaks()

normalised = subsets.apply(apply_normalisation)

In [ ]:
all_buffers.to_file('all_buffers.geojson', driver='GeoJSON')

In [ ]:
list(all_buffers)

In [ ]:
# Join the points to the buffers

buffered_points = gpd.sjoin(all_buffers, points, how="inner", op='intersects')